In [ ]:
from IPython.display import display, Markdown, HTML
import logging
import random

In [ ]:
# Suppress INFO messages from httpx as there can be distracting messages that aren't relevant to the user
logging.getLogger("httpx").setLevel(logging.ERROR)

class Chatbot:
    def __init__(self, model_name, system_role_content, max_inputs, tone, input_length_limit=4000):
        # set instance variables
        self.max_inputs = max_inputs
        self.input_length_limit = input_length_limit
        # Initialize history list to store the conversation history
        self.history = []
        # Initialize input count to keep track of the number of inputs
        self.input_count = 0

        # Set tone parameters by calling set_tone method defined below
        self.set_tone(tone)
        
        from class_version import OpenAIClient
        # Initialize the AI instance with the tone parameters
        self.ai = OpenAIClient(
            model_name=model_name,
            system_role_content=system_role_content,
            
            # use the values set by set_tone method 
            temperature=self.temperature,
            top_p=self.top_p
        )

    def set_tone(self, tone):
        
        # set temperature and top_p based on the tone by choosing random value within the given range
        if tone == "creative":
            self.temperature = random.uniform(0.8, 1.35)
            self.top_p = random.uniform(0.8, 1.0)
        elif tone == "balanced":
            # there is a buffer zone of .7 to .8 to create a defined difference between balanced and creative
            self.temperature = random.uniform(0.4, 0.7)
            self.top_p = random.uniform(0.4, 0.7)
        elif tone == "precise":
            # there is a buffer zone of .3 to .4 to create a defined difference between balanced and precise
            self.temperature = random.uniform(0.0, 0.3)
            self.top_p = random.uniform(0.0, 0.3)
        else:
            raise ValueError("Invalid tone. Choose from 'creative', 'balanced', or 'precise'.")
        
        
        
    def chat(self):
        # loop to continue the conversation until the max_inputs is reached or the user types "exit" or "quit"
        while self.input_count < self.max_inputs:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            if len(user_input.split()) > self.input_length_limit:
                display(Markdown(f"__Error: Input exceeds the length limit of {self.input_length_limit} words.__"))
                # skip the current iteration and continue with the next iteration
                continue

            # Display user prompt
            display(HTML(f'''
                        <article style="background-color: rgb(70, 70, 70); margin: 10px; padding: 10px; height: auto; 
                        line-height: 1.5; width: 75%"><b>You:</b> {user_input}</article>
                        '''))

            # Add user input to history
            self.history.append(f"User: {user_input}")

            # Create a prompt with the conversation history by using the join method to concatenate the list of prompts and responses
            prompt = "\n".join(self.history) + f"\nAI:"

            # Get the AI response
            response = self.ai.get_response(prompt)

            # Add AI response to history
            self.history.append(f"AI: {response}")

            # Increment the input count
            self.input_count += 1
            
            # Display AI response
            display(Markdown(f"__AI:__ {response}"))
            
            if self.input_count >= self.max_inputs:
              display(Markdown(f"### __This conversation has reached the limit of {self.max_inputs} inputs.__"))
              
        print("Saving file....") 
        print("This may take a few moments.")
        self.save_conversation()
        
        
        
    def save_conversation(self):
        from class_version import OpenAIClient
        self.ai = OpenAIClient(
            model_name='gpt-4o-mini',
            system_role_content="You are a helpful assistant.",
            temperature=0.1,
            top_p=0.1
        )
        name = self.ai.get_response(f"""The following is a record of a conversation between a user and an AI assistant: {self.history}
                            Please find a word or a phrase with which to name this conversation to be the name of the file.
                            The name should be no longer than three words and should not contain any special characters or spaces.""")
        
        # Define the directory path
        import os
        directory = "C:/Users/Rebecca/OneDrive/Documents/Python AI/AI Conversations"
        
        # Ensure the directory exists
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # Use the directory path when opening the file
        file_path = os.path.join(directory, name)
        
        try:
            with open(file_path + ".md", "w") as markdown:
                for message in self.history:
                    markdown.write(message + "\n")
                
                    
            with open(file_path + ".md", "r") as markdown:
                md_content = markdown.read()
            
            
            with open('html_formatting.txt', 'r') as html_instructions:  
                instructions = html_instructions.read()
            
                
                html_dev = OpenAIClient(model_name='gpt-4o-mini', 
                                        system_role_content="You are a skilled web developer.", 
                                        temperature=0.1, 
                                        top_p=0.1)
                
                import datetime
                # Current date and time
                date = datetime.datetime.now()

                # Format date and time
                formatted_date = date.strftime("%b %#d, %Y %#I:%M %p")
        
                conversation = html_dev.get_response(f"""Please convert {md_content} to HTML format preserving the markdown formatting it has.
                                        You can use the following code as a template: {instructions}. Pleaes put this as the date in the top right corner: {formatted_date}
                                        The file name should be displayed as an h2 header at the top of the page. This is the name: {name}
                                        """) 
                
            with open(file_path + ".html", "w") as file:
                # Trim the conversation to only include the actual HTML and not the response of the AI
                start_index = conversation.find("<!DOCTYPE html>")
                end_index = conversation.rfind("</html>") + len("</html>")
                conversation = conversation[start_index:end_index]
                file.write(conversation)
                
            # Verify the HTML file was created and has valid content
            if os.path.exists(file_path + ".html") and os.path.getsize(file_path + ".html") > 30:  # Threshold for meaningful content
                os.remove(file_path + ".md")  # Delete the .md file
                extension = ".html"
                
            else:
            # keep the markdown file if the HTML file is empty or has insufficient content
                extension = ".md"
                raise ValueError("HTML file is empty or has insufficient content.")
              
                
                  
        except Exception as e:
            print(f"Error: {e}")
        
        display(Markdown(f"### __The conversation has been saved as {name}.{extension}__"))
        
chatbot = Chatbot(
    model_name="gpt-4o-mini",
    system_role_content="""You are a tech expert. """,
    max_inputs=50,
    tone="balanced"
)

chatbot.chat()           



__AI:__ I'm just a computer program, but I'm here and ready to help you! How can I assist you today?

__AI:__ I'm best at providing information, answering questions, assisting with problem-solving, and offering suggestions across a wide range of topics. Whether you need help with research, writing, learning something new, or just want to chat, I'm here to assist! What do you need help with today?

__AI:__ **Abstract**

This research paper explores the inherent non-dependability of natural phenomena through the lens of the Naive Bayes algorithm, a fundamental statistical tool in machine learning. The Naive Bayes algorithm operates under the assumption that features are conditionally independent given the class label. This assumption, while often violated in real-world scenarios, serves as a critical framework for understanding the complexities of natural systems. By analyzing various datasets that represent natural processes, we demonstrate instances where the independence assumption leads to surprisingly accurate predictions, thereby highlighting the limitations of traditional models that rely on interdependent feature relationships. Our findings suggest that the apparent reliability of Naive Bayes in certain contexts underscores the unpredictable nature of complex systems, where underlying dependencies may be obscured. This paper contributes to the discourse on the unpredictability of nature by illustrating how machine learning models can yield insights even when foundational assumptions do not hold, ultimately questioning the extent to which we can depend on deterministic interpretations of natural phenomena. 

Keywords: Naive Bayes, feature independence, natural phenomena, machine learning, unpredictability.

__AI:__ Here’s a list of websites and resources that may discuss the topics related to the Naive Bayes algorithm, feature independence, and the unpredictability of natural phenomena:

1. **Google Scholar** (scholar.google.com)
   - A comprehensive database of scholarly articles where you can find research papers on Naive Bayes and its applications in various fields.

2. **ResearchGate** (www.researchgate.net)
   - A social networking site for researchers to share papers and results. You can find discussions and papers related to machine learning and Naive Bayes.

3. **arXiv** (arxiv.org)
   - A repository of preprint articles in various fields, including computer science and statistics. Search for papers related to Naive Bayes and feature independence.

4. **Kaggle** (www.kaggle.com)
   - A platform for data science competitions that often features discussions and notebooks using Naive Bayes in various datasets, including those related to natural phenomena.

5. **Towards Data Science** (towardsdatascience.com)
   - A Medium publication that features articles on data science, machine learning, and algorithms, including practical applications of Naive Bayes.

6. **Machine Learning Mastery** (machinelearningmastery.com)
   - A website that provides tutorials and explanations of machine learning concepts, including the Naive Bayes algorithm.

7. **StatQuest with Josh Starmer** (statquest.org)
   - A resource for understanding statistics and machine learning concepts, including Naive Bayes, through clear explanations and videos.

8. **Wikipedia** (en.wikipedia.org/wiki/Naive_Bayes_classifier)
   - The Wikipedia page on Naive Bayes provides a general overview of the algorithm, its assumptions, and applications.

9. **Cross Validated (Stack Exchange)** (stats.stackexchange.com)
   - A Q&A site for statistics and data analysis where you can find discussions on the assumptions of Naive Bayes and its implications.

10. **SpringerLink** (link.springer.com)
    - A database of scientific articles where you can find research on machine learning and its applications in understanding natural systems.

These resources should provide a solid starting point for exploring the relationship between the Naive Bayes algorithm, feature independence, and the unpredictability of nature.

__AI:__ Here’s a table of possible research ideas related to the non-dependability of nature and the Naive Bayes algorithm, along with their advantages and disadvantages:

| Research Idea                                      | Advantages                                                                                      | Disadvantages                                                                                 |
|----------------------------------------------------|------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------|
| 1. **Analyzing Naive Bayes in Environmental Data** | - Real-world application in ecology and environmental science. <br> - Potential for impactful findings. | - Data may be noisy or incomplete. <br> - Assumptions of independence may not hold.        |
| 2. **Comparative Study of Machine Learning Models** | - Provides a broader perspective on model performance. <br> - Highlights strengths and weaknesses of Naive Bayes. | - Requires extensive data collection and analysis. <br> - May involve complex model comparisons. |
| 3. **Feature Independence in Biological Systems**   | - Explores a fundamental question in biology. <br> - Can lead to insights in genetics or ecology. | - Biological systems are often highly interdependent. <br> - Difficult to isolate features.  |
| 4. **Impact of Assumption Violations on Predictions** | - Directly addresses the limitations of Naive Bayes. <br> - Can contribute to model improvement. | - May require advanced statistical techniques. <br> - Findings may be context-specific.     |
| 5. **Case Studies of Naive Bayes in Social Sciences** | - Expands the application of Naive Bayes beyond natural sciences. <br> - Engages with interdisciplinary research. | - Social phenomena may be too complex for Naive Bayes. <br> - Ethical considerations in data use. |
| 6. **Exploring Non-Linear Relationships in Nature** | - Investigates the complexity of natural systems. <br> - Could lead to the development of new models. | - Requires advanced statistical knowledge. <br> - May not directly relate to Naive Bayes.   |
| 7. **Simulating Natural Processes with Naive Bayes** | - Provides a controlled environment to test assumptions. <br> - Can lead to innovative insights. | - Simulations may not accurately reflect reality. <br> - Time-consuming to set up and analyze. |
| 8. **Naive Bayes for Predicting Climate Change Effects** | - Highly relevant and timely topic. <br> - Potential for significant societal impact. | - Complex interactions in climate data may challenge independence assumptions. <br> - Requires interdisciplinary knowledge. |

This table provides a structured overview of potential research ideas, helping you weigh the pros and cons of each option as you consider your focus for the research paper.

__AI:__ The idea of exploring the non-dependability of nature through the lens of the Naive Bayes algorithm and its assumptions has been touched upon in various research contexts, but the specific angle you are considering may not have been extensively covered. Here are some points to consider regarding existing literature:

1. **Naive Bayes in Environmental Science**: There are studies that apply the Naive Bayes algorithm to environmental data, but they often focus on classification tasks rather than the philosophical implications of feature independence.

2. **Feature Independence in Biological Systems**: Research in biology often discusses the interdependence of features, particularly in genetics and ecology. However, studies specifically addressing the implications of using Naive Bayes in these contexts may be limited.

3. **Comparative Studies of Machine Learning Models**: Many papers compare various machine learning models, including Naive Bayes, but they may not specifically address the philosophical implications of the independence assumption.

4. **Case Studies in Social Sciences**: While Naive Bayes has been applied in social science research, discussions about the implications of its assumptions on the predictability of social phenomena are less common.

5. **Simulations and Non-Linear Relationships**: Research that simulates natural processes or explores non-linear relationships often involves more complex models than Naive Bayes, which may lead to less focus on the independence assumption.

To determine if your specific research idea has been written about before, I recommend conducting a thorough literature review using academic databases like Google Scholar, ResearchGate, and arXiv. Look for keywords related to your topic, such as "Naive Bayes," "feature independence," "natural phenomena," and "machine learning in ecology." This will help you identify existing studies and determine how your research can contribute to the field.

__AI:__ Goodbye! If you have more questions in the future or need assistance, feel free to reach out. Have a great day!

### __The conversation has been saved as ResearchNaiveBayes.html.__